# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cleancity_data_df=pd.read_csv('../output_data/CityData.csv')
cleancity_data_df.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,bluff,-46.6000,168.3333,55.80,70,83,7.52,NZ,1627520465
1,agadir,30.4202,-9.5982,68.45,78,9,3.11,MA,1627520465
2,kapaa,22.0752,-159.3190,87.78,70,92,5.01,US,1627520465
3,shancheng,34.7953,116.0817,68.94,97,100,19.39,CN,1627520466
4,buenavista,8.9769,125.4089,85.30,64,94,2.28,PH,1627520466


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps & Create Location Variables 
gmaps.configure(api_key=g_key)

locations = cleancity_data_df[["Lat", "Lng"]]
humidity = cleancity_data_df["Humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=300, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
narrow_city_df = cleancity_data_df.loc[(cleancity_data_df["Max Temp"] < 80) & (cleancity_data_df["Max Temp"] > 70) \
                                    & (cleancity_data_df["Wind Speed"] < 10) \
                                    & (cleancity_data_df["Cloudiness"] == 0)].dropna()
narrow_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
73,siniscola,40.5719,9.6922,72.95,67,0,3.89,IT,1627520479
83,jacareacanga,-6.2222,-57.7528,77.09,44,0,2.42,BR,1627520481
96,westport,41.1415,-73.3579,74.86,85,0,7.00,US,1627520371
203,poirino,44.9208,7.8469,73.18,90,0,0.47,IT,1627520509
239,bethel,41.3712,-73.4140,72.12,79,0,3.40,US,1627520473
290,makat,47.6469,53.3463,72.79,36,0,9.42,KZ,1627520530
296,tortoli,39.9256,9.6572,75.81,56,0,5.03,IT,1627520532
303,alta floresta,-9.8756,-56.0861,73.56,49,0,5.03,BR,1627520365
304,abu samrah,35.3029,37.1841,75.56,46,0,8.16,SY,1627520533
309,karacakoy,41.4029,28.3834,78.84,80,0,2.73,TR,1627520535


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [5]:
hotel_df = narrow_city_df[["City", "Country", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
73,siniscola,IT,40.5719,9.6922,
83,jacareacanga,BR,-6.2222,-57.7528,
96,westport,US,41.1415,-73.3579,
203,poirino,IT,44.9208,7.8469,
239,bethel,US,41.3712,-73.4140,
290,makat,KZ,47.6469,53.3463,
296,tortoli,IT,39.9256,9.6572,
303,alta floresta,BR,-9.8756,-56.0861,
304,abu samrah,SY,35.3029,37.1841,
309,karacakoy,TR,41.4029,28.3834,


In [6]:
MainUrl = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params   = {"radius": 5000,"type": "lodging","key": g_key,}

for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    print(f"Retrieving Results for Index {index}: {row['City']}.")
    response = requests.get(MainUrl, params=params).json()   
    results  = response['results']
    try:
        print(f"Closest hotel is {results[0]['name']}.")
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']    
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
    print("------------")

Retrieving Results for Index 73: siniscola.
Closest hotel is Hotel Sa Tanca E Bore.
------------
Retrieving Results for Index 83: jacareacanga.
Closest hotel is Casa do Ian😎.
------------
Retrieving Results for Index 96: westport.
Missing field/result... skipping.
------------
Retrieving Results for Index 203: poirino.
Closest hotel is Hotel Brindor.
------------
Retrieving Results for Index 239: bethel.
Missing field/result... skipping.
------------
Retrieving Results for Index 290: makat.
Missing field/result... skipping.
------------
Retrieving Results for Index 296: tortoli.
Missing field/result... skipping.
------------
Retrieving Results for Index 303: alta floresta.
Closest hotel is Floresta Amazonica Hotel.
------------
Retrieving Results for Index 304: abu samrah.
Missing field/result... skipping.
------------
Retrieving Results for Index 309: karacakoy.
Missing field/result... skipping.
------------
Retrieving Results for Index 320: balsas.
Closest hotel is Novo Hotel.
------

In [7]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
73,siniscola,IT,40.5719,9.6922,Hotel Sa Tanca E Bore
83,jacareacanga,BR,-6.2222,-57.7528,Casa do Ian😎
96,westport,US,41.1415,-73.3579,
203,poirino,IT,44.9208,7.8469,Hotel Brindor
239,bethel,US,41.3712,-73.4140,
290,makat,KZ,47.6469,53.3463,
296,tortoli,IT,39.9256,9.6572,
303,alta floresta,BR,-9.8756,-56.0861,Floresta Amazonica Hotel
304,abu samrah,SY,35.3029,37.1841,
309,karacakoy,TR,41.4029,28.3834,


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [12]:
# Add marker layer ontop of heat map

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Display figure
fig




Figure(layout=FigureLayout(height='420px'))